# **Lab 3: Iterative methods**
**Dániel Szabó**

# **Abstract**

This laboratory is mostly about approximately solving a system of linear equations given in the form $Ax=b$ using iterative methods. The mandatory tasks are the implementation of the Jacobi method and the Gauss-Seidel method for solving system $Ax=b$, and Newton's method for finding a root of a nonlinear function $f: \mathbb{R}\to\mathbb{R}$.

#**About the code**

In [31]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2021 Dániel Szabó (dszabo@kth.se)

# This file is part of the course DD2365 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [32]:
# Load neccessary modules.
import numpy as np
import random

random.seed(1)

# **Introduction**

Some details about the solved problems are presented here.

1. Jacobi iteration for $Ax=b$: The input is a real, quadratic matrix $A\in\mathbb{R}^{n×n}$ and a real vector $b\in\mathbb{R}^n$. The task is to approximate $x\in\mathbb{R}^n$ that satisfies $Ax=b$ by performing iterations $x^{(k+1)}=(I-D^{-1}A)x^{(k)}+D^{-1}b$ where $D\in\mathbb{R}^{n×n}$ is the diagonal version of $A$ (i.e. $\forall i,j\in[n]\ d_{ij}=a_{ii}$ if $i=j$, otherwise $d_{ij}=0$).

2. Gauss-Seidel iteration for $Ax=b$: The input is a real, quadratic matrix $A\in\mathbb{R}^{n×n}$ and a real vector $b\in\mathbb{R}^n$. The task is to approximate $x\in\mathbb{R}^n$ that satisfies $Ax=b$ by performing iterations $x^{(k+1)}=(I-L^{-1}A)x^{(k)}+L^{-1}b$ where $L\in\mathbb{R}^{n×n}$ is the lower triangular version of $A$ (i.e. $\forall i,j\in[n]\ l_{ij}=0$ if $i<j$, otherwise $l_{ij}=a_{ij}$).

3. Newton's method for scalar nonlinear equation $f(x)=0$: The input is a real function $f:\mathbb{R}\to\mathbb{R}$. The task is to approximate a root of $f$, i.e. $x\in\mathbb{R}$ such that $f(x)=0$, by performing iterations $x^{(k+1)}=x^{(k)}-f(x^{(k)})/f'(x^{(k)})$ where function $f'$ is the derivative of $f$.

# **Method**

Input validation: In most of the tasks we check if the input matrix is actually a matrix, i.e. if it has the same number of elements in each row, and it is done by the matrix_check function. Furthermore, in each task we also check if the input satisfies some other requirements, e.g. if the input matrix is a square matrix, if it is compatible with another input or if the precision parameter is positive.

In [33]:
def matrix_check(M):
    for i in range(len(M)):
        if(len(M[i])!=len(M[0])):
            raise Exception("The number of elements in each row of a matrix should be the same.")
    return

1. Jacobi iteration for $Ax=b$: First, the validity of the input is checked. Then, the diagonal version $D$ of $A$ and $D$'s inverse is calculated. We check if the convergence criterion $||I-D^{-1}A||<1$ is satisfied: if it is not, we cannot continue so an exception is raised. Otherwise, we perform the iterations described in Example 7.8 of the lecture notes (that is, $x^{(k+1)}=(I-D^{-1}A)x^{(k)}+D^{-1}b$) until residual $||Ax-b||$ is smaller than the input precision parameter.

In [34]:
def jacobi_iteration(A, b, eps):
    matrix_check(A)
    n = len(A)
    if n != len(A[0]):
        raise Exception("The input matrix should be a square matrix.")
    if n != len(b):
        raise Exception("The number of elements in the input vector should be the same as the size of the matrix.")
    if eps<=0:
        raise Exception("Epsilon should be positive.")
    
    D = np.diag(np.diag(A))
    Dinv = np.linalg.inv(D)
    if np.linalg.norm(np.subtract(np.eye(n), Dinv @ A)) >= 1:
        raise Exception("convergence criterion not fulfilled")
    x = np.zeros(n)
    residual = np.inf
    while residual > eps:
        x = np.add(np.dot(np.subtract(np.eye(n), Dinv @ A), x), Dinv @ b)
        residual = np.linalg.norm(np.subtract(A @ x, b))
    return x

2. Gauss-Seidel iteration for $Ax=b$: First, the validity of the input is checked. Then, the lower triangular version $L$ of $A$ and $L$'s inverse is calculated. We check if the convergence criterion $||I-L^{-1}A||<1$ is satisfied: if it is not, we cannot continue so an exception is raised. Otherwise, we perform the iterations described in Example 7.9 of the lecture notes (that is, $x^{(k+1)}=(I-L^{-1}A)x^{(k)}+L^{-1}b$) until residual $||Ax-b||$ is smaller than the input precision parameter.

In [35]:
def gauss_seidel_iteration(A, b, eps):
    matrix_check(A)
    n = len(A)
    if n != len(A[0]):
        raise Exception("The input matrix should be a square matrix.")
    if n != len(b):
        raise Exception("The number of elements in the input vector should be the same as the size of the matrix.")
    if eps<=0:
        raise Exception("Epsilon should be positive.")
    
    L = np.tril(A)
    Linv = np.linalg.inv(L)
    if np.linalg.norm(np.subtract(np.eye(n), Linv @ A)) >= 1:
        raise Exception("convergence criterion not fulfilled")
    x = np.zeros(n)
    residual = np.inf
    while residual > eps:
        x = np.add(np.dot(np.subtract(np.eye(n), np.dot(Linv, A)), x), np.dot(Linv, b))
        residual = np.linalg.norm(np.subtract(np.dot(A, x), b))
    return x

3. Newton's method for scalar nonlinear equation $f(x)=0$: Let us suppose that $f$ is a polynomial, because this way, it is easy to calculate its derivative and also its value at a given point. This simplification can be justified by the fact that every infinitely differentiable function can be approximated by a polynomial, e.g. one of its Taylor polynomials. If $f$ is an $n^\textrm{th}$-degree polynomial, then it can be given by an array containing its $n+1$ coefficients: if $f(x)=x_0+x_1x+\dots+x_nx^n$, then the array is $[x_0,x_1,\dots,x_n]$.

  We have two auxiliary methods. The one named "f" calculates the value of a polynomial given by the array of its coefficients at a given point. The "derivative" calculates the derivative of a polynomial given by the array of its coefficients, so the result is another polynomial, also represented by its array of coefficients.

  The main method ("newton_scalar") follows Algorithm 8.2 of the lecture notes. But instead of receiving an initial guess $x_0$ as an input, it generates some (at most ten) numbers randomly. For a certain $x_0$, we perform the iterations $x^{(k+1)}=x^{(k)}-f(x^{(k)})/f'(x^{(k)})$ until $f(x)$ is close enough to 0. We can also stop doing the iterations if we have already been doing it for a lot of time ($100+1/\varepsilon$ steps), because the function may not have a real root at any point reachable by performing the iteration's steps from $x_0$. If we get a result for an $x_0$, we return it, otherwise an exception is raised, because in this case, with a high probability, the function does not have any real roots.

In [36]:
def f(coeff, x):
    result = 0
    n = len(coeff)-1
    for i in range(n+1):
        result += coeff[i] * (x**i)
    return result

def derivative(coeff):
    n = len(coeff)-1
    der = []
    for i in range(n):
        der.append(coeff[i+1]*(i+1))
    return der

def newton_scalar(coeff, eps):
    if eps<=0:
        raise Exception("Epsilon should be positive.")
    for _ in range(10):
        x0 = random.randint(-50, 50)
        x = x0
        fx = f(coeff, x)
        limit = 100+1/eps
        while np.abs(fx) > eps and limit > 0:
            der = derivative(coeff)
            df = f(der, x)
            x -= fx/df
            fx = f(coeff, x)
            limit -= 1
        if limit > 0:
            return x
    raise Exception("Couldn't find any root")

# **Results**

The verification of the methods is done by computing the results for some random test data using the algorithms of the Method section and then calculating the error metrics that are required in the lab instructions using some numpy methods. Then we check if these values are close to 0. Some tests with invalid input are also provided.

Since the inputs are generated randomly, it is possible that sometimes the Jacobi or the Gauss-Seidel method does not converge, or that Newton's method does not find a real root of the input function. In these cases, an exception is raised, and the corresponding test should be run again. Hopefully, there will be no need for too many experiments as the inputs are generated in a way so that the methods can work with them with high probability.

1. Jacobi iteration for $Ax=b$: We generate a random matrix $A$ that is diagonally dominant with high probability, because we know that $A$ being diagonally dominant is a sufficient condition for the method to be convergent. A random vector $b$ is generated as well, and the approximate solution is calculated with our method. Then we check if norms $||Ax-b||$ and $||x-y||$ are close to zero, where $y=A^{-1}b$. Invalid input handling is also tested.

In [37]:
n = random.randint(2,5)
epsilon = 10**-5
A = np.array([[random.gauss(0, 1) for _ in range(n)] for __ in range(n)])
for i in range(n):
    A[i,i] = random.gauss(0, 10)
b = [random.gauss(0, 5) for _ in range(n)]
x = jacobi_iteration(A,b, epsilon)
print("A =", A)
print("b =", b)
print("x =", x)

norm1 = np.linalg.norm(np.subtract(np.dot(A,x), b))
norm2 = np.linalg.norm(np.subtract(x, np.dot(np.linalg.inv(A), b)))
print("errors:", norm1, norm2)
np.testing.assert_almost_equal(norm1, 0, decimal=np.log10(1/epsilon)-1)
np.testing.assert_almost_equal(norm2, 0, decimal=np.log10(1/epsilon)-1)

# Invalid input test: not compatible inputs
np.testing.assert_raises(Exception, jacobi_iteration, np.array([[1,2],[3,4]]), np.array([1,2,3]))
# Invalid input test: non-square matrix
np.testing.assert_raises(Exception, jacobi_iteration, np.array([[1,2],[3,4],[5,6]]), np.array([1,2]))

A = [[  1.61261773  -0.75843985   0.46207506]
 [  0.1934651  -10.52647335  -1.12791503]
 [ -0.49932479   0.47117408   8.72957704]]
b = [0.830016173448243, -7.6694178766028145, -0.9214567588221791]
x = [ 0.89712791  0.75525164 -0.09500515]
errors: 4.965039609223026e-06 2.691781159671771e-06


2. Gauss-Seidel iteration for $Ax=b$: We generate a random matrix $A$ that is diagonally dominant with high probability, because we know that $A$ being diagonally dominant is a sufficient condition for the method to be convergent. A random vector $b$ is generated as well, and the approximate solution is calculated with our method. Then we check if norms $||Ax-b||$ and $||x-y||$ are close to zero, where $y=A^{-1}b$. Invalid input handling is also tested.

In [38]:
n = random.randint(2,5)
epsilon = 10**-5
A = np.array([[random.gauss(0, 1) for _ in range(n)] for __ in range(n)])
for i in range(n):
    A[i,i] = random.gauss(0, 10)
b = [random.gauss(0, 5) for _ in range(n)]
x = gauss_seidel_iteration(A,b, epsilon)
print("A =", A)
print("b =", b)
print("x =", x)

norm1 = np.linalg.norm(np.subtract(np.dot(A,x), b))
norm2 = np.linalg.norm(np.subtract(x, np.dot(np.linalg.inv(A), b)))
print("errors:", norm1, norm2)
np.testing.assert_almost_equal(norm1, 0, decimal=np.log10(1/epsilon)-1)
np.testing.assert_almost_equal(norm2, 0, decimal=np.log10(1/epsilon)-1)

# Invalid input test: not compatible inputs
np.testing.assert_raises(Exception, gauss_seidel_iteration, np.array([[1,2],[3,4]]), np.array([1,2,3]))
# Invalid input test: non-square matrix
np.testing.assert_raises(Exception, gauss_seidel_iteration, np.array([[1,2],[3,4],[5,6]]), np.array([1,2]))

A = [[ 1.98791248  0.20296918]
 [-0.14470231  9.09031026]]
b = [-1.827721331162584, 1.0908590668029121]
x = [-0.93015805  0.10519586]
errors: 5.649281353825586e-08 2.8375640892033392e-08


3. Newton's method for scalar nonlinear equation $f(x)=0$: Function "print_poly" prints a polynomial given by its coefficients in a readable way. The degree of the input polynomial is a random integer between 2 and 7. The coefficients are also generated randomly. The exact roots (array $y$) are calculated with the numpy roots function, and one of the real roots ($x$) is computed with our function. Values $|f(x)|$ and $\min_i|x-y_i|$ are calculated (the latter is the difference of the approximate root and the corresponding exact root), and we check if they are close to zero. Invalid input handling ($\varepsilon\le0$) is also tested.

In [39]:
def print_poly(coeffs):
    print(coeffs[0], end='')
    for i in range(1,len(coeffs)):
        print(" + ",coeffs[i]," * x^",i, end='', sep='')
    print("")

n = random.randint(2,7)
epsilon = 10**-5
poly = [random.gauss(0, 1) for _ in range(n+1)]
print_poly(poly)
y = np.roots(poly[::-1])
print("y =",y)
x = newton_scalar(poly, epsilon)
print("x =",x)

err1 = np.abs(f(poly, x))
err2 = np.min(abs(y-x))
print("errors:", err1, err2)
np.testing.assert_almost_equal(err1, 0, decimal=np.log10(1/epsilon)-1)
np.testing.assert_almost_equal(err2, 0, decimal=np.log10(1/epsilon)-1)

# Invalid input test: non-positive epsilon
np.testing.assert_raises(Exception, newton_scalar, [1,2,2,4], 0)

1.024288678487018 + -1.673089745800391 * x^1 + -0.29340299027935385 * x^2
y = [-6.26003573  0.55767481]
x = 0.5576748060907004
errors: 9.781203624825707e-12 4.8896442450541144e-12


# **Discussion**

The results were exactly as expected. All the methods implemented for solving the tasks, actually succeeded in solving them, as it is confirmed by the test results. Furthermore, the inputs are validated properly.